In [93]:
import os
import math
import random
import itertools as it
import numpy as np
import concurrent.futures as cf
from scipy import stats
from scipy.stats.mstats import gmean
from statsmodels.stats import multitest as mt
from sklearn.decomposition import PCA as skPCA
pca = skPCA(n_components=3)
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
sns.set()

In [9]:
def mysplit(c):
    tl = c.lstrip('0123456789')
    hd = c[:len(c)-len(tl)]
    return(hd,tl)
def readMatrix(fpath,offset=False,iglines = 0,split=False):
    dicto = {}
    with open(fpath,'r') as f:
        allc = [l.strip().split('\t') for l in f.readlines()]
    #if '*' in allc[0][0]:
    #    hdr = [tuple(e.split('*')) for e in allc[0]]
    #else:
    #    hdr = [tuple(e.split('_')) for e in allc[0]]
    allc = allc[iglines:]
    hdr = allc[0]
    if split:
        hdr = [mysplit(e[1:]) for e in hdr]
    if offset:
        hdr = hdr[1:]
    dicto ={h:{} for h in hdr}
    for l in allc[1:]:
        for i,h in enumerate(hdr):
            dicto[h][l[0]] = l[i+1]
        #for l in allc[1:]:
    #    dicto[l[0]] = {hdr[i]:l[i+1] for i in range(len(hdr))}
    return dicto

In [10]:
def sampleCounts(dicto,firstN):
    bigList = []
    names=[]
    counts = []
    if sum(dicto.values())<firstN:
            return None
    for g in dicto:
        for j in range(dicto[g]):
            bigList.append(g)
        #names.append(r)
        #counts.append(dicto[r])
    #for i,count in enumerate(counts):
    #    for j in range(count):
    #        bigList.append(names[i])
    #if len(bigList)<firstN:
    #    return None
    if len(bigList)!=sum(dicto.values()):
        print("MISMATCH")
    random.shuffle(bigList)
    subList = bigList[:firstN]
    retDict = {k:0 for k in dicto}
    for elt in subList:
        retDict[elt] += 1
    return retDict

In [11]:
def plotPCA(ax,counts,samples,groups,colors,anno=False):
    res = pca.fit_transform(counts)
    print(len(res))
    for i,s in enumerate(samples):
        ax.scatter(res[i][0], res[i][1],color=colors[groups[i]],alpha=0.75,edgecolors='none')#,s=100)#plt.cm.spectral(cond_idx[conditions[i]]), alpha=0.7)
    if anno:
        for i,s in enumerate(samples):
            ax.annotate(s, xy = (res[i][0], res[i][1]), xytext = (4, 4), textcoords = 'offset points')
    
    handles = []
    for grp in set(groups):
        handles.append(mpatches.Patch(color=colors[grp],label=grp))
    
    expvar = pca.explained_variance_ratio_
    ax.set_xlabel('PC1: {:.2f}% variance explained'.format(100*expvar[0]))
    ax.set_ylabel('PC2: {:.2f}% variance explained'.format(100*expvar[1]))
    
    return handles
    #ax.legend(handles=handles,prop={'size':8},loc='best')
    #plt.tight_layout()
    #plt.show()


In [123]:
#Directory definitions
#indir: contains count data, e.g. from FeatureCounts, for all samples (matrix for riboseq/rnaseq same as for ribo/PLATEseq)
indir = '../data/'
#outdir: output directory for figures
outdir = '../figs/'
#wDir: working directory containing DESeq2 results, normalized counts ranked lists, gene sets, and other text files 
wDir = '../workingdir/'
#gsDir: working directory containing GSEA output
gsdir = '../gsea/'
#refDir: source directory for genomic reference information not provided in working directory
refDir = '../ReferenceInfo/'
#gspath: path to GSEA CLI
gspath = '/home/ubuntu/datadrive/riboPLATEseq/GSEA_4.1.0/'

In [28]:
#Non-polyadenylated genes; mostly identifies histones, snoRNAs, and scaRNAs, among other ncRNA
#From Yang et al 2011, "Genomewide characterization of non-polyadenylated RNAs" (https://doi.org/10.1186/gb-2011-12-2-r16)
#These genes were originally identified with symbols from hg19 (UCSC "Known Genes" predictor); conversion of unrecognized 
#loci to hg38-recognized GENCODE symbols required searching NCBI Nucleotide for ~100 source records
with open(refDir+"genes_non_pA.txt",'r') as f:
    badG = [e.strip() for e in f.readlines()]
print(len(badG))

464


In [118]:
#Non-coding gene set determined from RefSeq GTF annotation
ncGenes = set()
cGenes = set()

with open(refDir+"GRCh38_refseq_ERCC.gtf",'r') as f:
    for line in f:
        try:
            gtfE = line.strip().split('\t')[-1].strip().split(';')
            nmDict[gtfE[1].strip().split(' ')[1].strip('"').split('.')[0]]=gtfE[0].strip().split(' ')[1].strip('"')
            ent = line.split('\t')[-1].split(';')
            elt = ent[1].split()[1]
            if 'NR_' in elt:
                ncGenes.add(ent[0].split()[1].strip('"'))
            elif 'NM_' in elt:
                cGenes.add(ent[0].split()[1].strip('"'))
        except Exception:
            pass
print(len(ncGenes),len(cGenes))
qcGenes = ncGenes & cGenes
ncGenes = ncGenes - cGenes

print(len(ncGenes),len(qcGenes))

13611 22085
10334 3277


In [15]:
#TOP gene candidate transcripts (NCBI RefSeq; "NM_") from Yamashita et al. 2011, "Comprehensive detection of human terminal
#oligo-pyrimidine (TOP) genes and analysis of their characteristics" (doi: https://dx.doi.org/10.1093%2Fnar%2Fgkn248)
#Specific subset of candidate transcripts chosen have homologues in the mouse genome that also contain TOP motifs
with open(refDir+"candidateTOP.txt",'r') as f:
    allC = [l.split('\t') for l in f.readlines()[1:]]
    topC = [l[0].split(',') for l in allC]
    tpc = []
    for c in topC:
        for ci in c:
            tpc.append(ci)
    tpc = list(set(tpc))
    
    topm = [l[0].split(',') for l in allC if l[6]!='0\n']
    tpcm = []
    for c in topm:
        for ci in c:
            tpcm.append(ci)
    tpcm = list(set(tpcm))
print(len(tpc),len(tpcm))

#Classical TOP gene set, defined in other studies but also annotated in Yamashita et al
with open(refDir+"classicalTOP.txt",'r') as f:
    classicTOP = [l.strip() for l in f.readlines()]
print(len([t for t in tpc if t in nmDict]),len([t for t in tpcm if t in nmDict]))



2222 309
2070 291


In [65]:
barcode_files = ['PLATE','riboPLATE','PLATE_ERCC','riboPLATE_ERCC']
bcDict = {}
for bc_pfx in barcode_files:
    bc_fl = 'barcodes_{}.tsv'.format(bc_pfx)
    with open(indir+bc_fl,'r') as f:
        f.readline()
        allbc = [l.strip().split('\t') for l in f.readlines()]
    print(bc_pfx,len(allbc))
    bcDict[bc_pfx] = {int(l[0]):(l[1],l[2]) for l in allbc}
    print(len(bcDict[bc_pfx]))


PLATE 96
96
riboPLATE 96
96
PLATE_ERCC 96
96
riboPLATE_ERCC 96
96


In [107]:
#maplist: vectorized representation of conditions in riboPLATE-seq study by barcode index
maplist = ['AZD', 'CTRL', 'MNK1', 'CTRL', 'PP242', 'CTRL', 'BKM', 'MNK1+BKM', 'CTRL', 'CTRL', 'MNK1+BKM', '4EGI1', 
           'CTRL', 'AZD', 'CTRL', 'MNK1', 'CTRL', 'PP242', 'CTRL', 'BKM', 'MNK1+BKM', 'MNK1+BKM', 'CTRL', 'CTRL', 
           'CTRL', 'CTRL', 'AZD', 'CTRL', 'MNK1', 'CTRL', 'PP242', 'CTRL', 'BKM', 'CTRL', 'MNK1+BKM', 'MNK1+BKM', 
           'PP242+MNK1', 'CTRL', 'CTRL', 'AZD', 'CTRL', 'MNK1', 'CTRL', 'PP242', 'CTRL', 'BKM', 'CTRL', 'CTRL',
           'CTRL', 'PP242+MNK1', 'PP242+MNK1', 'CTRL', 'AZD', 'CTRL', 'MNK1', 'CTRL', 'PP242', 'CTRL', 'BKM', '4EGI1', 
           'PP242+BKM', 'CTRL', 'CTRL', 'PP242+MNK1', 'PP242+MNK1', 'AZD', 'CTRL', 'MNK1', 'CTRL', 'PP242', 'CTRL', 'BKM',
           'CTRL', 'PP242+BKM', 'PP242+BKM', 'CTRL', '4EGI1', 'PP242+MNK1', 'CTRL', '4EGI1', 'CTRL', '4EGI1', 'CTRL', '4EGI1',
           'CTRL', 'CTRL', 'CTRL', 'PP242+BKM', 'PP242+BKM', 'CTRL', 'CTRL', 'PP242+BKM', 'CTRL', 'CTRL', 'CTRL', 'CTRL']

In [104]:
#rDict: readDict; holds {sample:{gene:counts for gene in genes} for sample in samples} 
#for each set of samples pooled in each (ribo)PLATE library
#barcodes are converted to numeric indices (0-95) for ease of reading/addressing individual wells
rDict = {}
#cdDict: holds {index:sample condition} for each (ribo)PLATE library 
cdDict = {}
for ct in ['riboPLATE','PLATE','riboPLATE_ERCC','PLATE_ERCC']:
    tmp = readMatrix(indir+ct+'.counts.txt')
    rDict[ct] = {k:{g:int(v) for (g,v) in tmp[k].items()} for k in tmp}
    bcs = bcDict[ct]
    bc_idx = {bcs[b][0]:b for b in bcs}
    rDict[ct] = {bc_idx[k]:rDict[ct][k] for k in rDict[ct]}
    #index to condition for each barcode in study
    cdDict[ct] = {bc:bcs[bc][1] for bc in bcs}

In [101]:
fullseq = {}
for lbl in ['RPF','RNA']:
    tmp = readMatrix(indir+lbl+'.counts.txt')
    fullseq.update({tuple(k.split('.')):{g:int(v) for (g,v) in tmp[k].items()} for k in tmp})
    print(len(fullseq))

10
23


In [110]:
#Setup for FINAL DESeq2 on riboPLATE -- excludes outliers, excludes ERCC
wDir = '/home/ubuntu/datadrive/riboPLATEseq/riboPLATEseq_repo/workingdir/'
badS = [0,4,6,9,10,11,22,34,36,60,76]

sampleIdx = [i for i in range(96)]
bc_rb = bcDict['riboPLATE']#[bcDict['riboPLATE'][i][0] for i in sampleIdx]
bc_rn = bcDict['PLATE']#[bcDict['PLATE'][i][0] for i in sampleIdx]
conditions = list(set(maplist))
conditions = [''.join(e.split('+')) for e in conditions]
conditionDict = {c:[i for i in range(96) if ''.join(bc_rb[i][1].split('+'))==c and ''.join(bc_rn[i][1].split('+'))==c] for c in conditions}#                    for [i for i in range(96) if ''.join(maplist[i].split('+'))==c] for c in conditions}
rb = 'riboPLATE'
rn = 'PLATE'
gsub = [g for g in goodG if 'ERCC-' not in g]
#gSums_rb = {g:sum([rDict[rb][i][g] for i in rDict[rb]]) for g in gsub}
#gSums_rn =  {g:sum([rDict[rn][i][g] for i in rDict[rn]]) for g in gsub}
lbl = 'final'
pfx = rb+'.'+lbl
print(pfx)
clFl = '{}.cols.txt'.format(pfx)
ctFl = '{}.cts.txt'.format(pfx)
samples = []
hdr = []
print(ctFl,clFl)
with open(wDir+ctFl,'w') as o, open(wDir+clFl,'w') as col:
    col.write('condition,type\n')
    for cond in conditions:
        for s in conditionDict[cond]:
            if s in badS: continue
            col.write("ribo_{},{},RIBO\n".format(s,cond))
            hdr.append("ribo_{}".format(s))
            samples.append(s)
    for cond in conditions:
        for s in conditionDict[cond]:
            if s in badS: continue
            col.write("rna_{},{},RNA\n".format(s,cond))
            hdr.append("rna_{}".format(s))

    print(len(samples),len(hdr))
    o.write('\t'.join(hdr)+'\n')
    #o.write("\t".join(['ribo_{}'.format(s) for s in samples])+'\n')
    for g in gsub:
        o.write("{}\t{}\t{}\n".format(g,'\t'.join([str(int(rDict[rb][i][g])) for i in samples]),'\t'.join([str(int(rDict[rn][i][g])) for i in samples])))


riboPLATE.final
riboPLATE.final.cts.txt riboPLATE.final.cols.txt
85 170


In [111]:
#Setup for DESeq2 on riboPLATE with outliers
#wDir = '/home/ubuntu/datadrive/riboPLATEseq/riboPLATEseq_repo/workingdir/'

sampleIdx = [i for i in range(96)]
bc_rb = bcDict['riboPLATE']#[bcDict['riboPLATE'][i][0] for i in sampleIdx]
bc_rn = bcDict['PLATE']#[bcDict['PLATE'][i][0] for i in sampleIdx]
conditions = list(set(maplist))
conditions = [''.join(e.split('+')) for e in conditions]
conditionDict = {c:[i for i in range(96) if ''.join(bc_rb[i][1].split('+'))==c and ''.join(bc_rn[i][1].split('+'))==c] for c in conditions}#                    for [i for i in range(96) if ''.join(maplist[i].split('+'))==c] for c in conditions}
rb = 'riboPLATE'
rn = 'PLATE'
gsub = [g for g in goodG if 'ERCC-' not in g]
#gSums_rb = {g:sum([rDict[rb][i][g] for i in rDict[rb]]) for g in gsub}
#gSums_rn =  {g:sum([rDict[rn][i][g] for i in rDict[rn]]) for g in gsub}
lbl = 'all'
pfx = rb+'.'+lbl
print(pfx)
clFl = '{}.cols.txt'.format(pfx)
ctFl = '{}.cts.txt'.format(pfx)
samples = []
hdr = []
print(ctFl,clFl)
with open(wDir+ctFl,'w') as o, open(wDir+clFl,'w') as col:
    col.write('condition,type\n')
    for cond in conditions:
        for s in conditionDict[cond]:
            col.write("ribo_{},{},RIBO\n".format(s,cond))
            hdr.append("ribo_{}".format(s))
            samples.append(s)
    for cond in conditions:
        for s in conditionDict[cond]:
            col.write("rna_{},{},RNA\n".format(s,cond))
            hdr.append("rna_{}".format(s))

    print(len(samples),len(hdr))
    o.write('\t'.join(hdr)+'\n')
    for g in gsub:
        o.write("{}\t{}\t{}\n".format(g,'\t'.join([str(int(rDict[rb][i][g])) for i in samples]),'\t'.join([str(int(rDict[rn][i][g])) for i in samples])))


riboPLATE.all
riboPLATE.all.cts.txt riboPLATE.all.cols.txt
96 192


In [113]:
#Setup for DESeq2 on ERCC dataset
#wDir = '/home/ubuntu/datadrive/riboPLATEseq/riboPLATEseq_repo/workingdir/'

sampleIdx = [i for i in range(96)]
bc_rb = bcDict['riboPLATE_ERCC']#[bcDict['riboPLATE'][i][0] for i in sampleIdx]
bc_rn = bcDict['PLATE_ERCC']#[bcDict['PLATE'][i][0] for i in sampleIdx]
conditions = list(set(cdDict['PLATE_ERCC'].values()))
conditions = [''.join(e.split('+')) for e in conditions]
conditionDict = {c:[i for i in range(96) if ''.join(bc_rb[i][1].split('+'))==c and ''.join(bc_rn[i][1].split('+'))==c] for c in conditions}#                    for [i for i in range(96) if ''.join(maplist[i].split('+'))==c] for c in conditions}
rb = 'riboPLATE_ERCC'
rn = 'PLATE_ERCC'
gsub = [g for g in goodG]
lbl = 'final'
pfx = rb+'.'+lbl
print(pfx)
clFl = '{}.cols.txt'.format(pfx)
ctFl = '{}.cts.txt'.format(pfx)
samples = []
hdr = []
print(ctFl,clFl)
with open(wDir+ctFl,'w') as o, open(wDir+clFl,'w') as col:
    col.write('ERCC,type\n')
    for cond in conditions:
        for s in conditionDict[cond]:
            col.write("ribo_{},{},RIBO\n".format(s,cond))
            hdr.append("ribo_{}".format(s))
            samples.append(s)
    for cond in conditions:
        for s in conditionDict[cond]:
            col.write("rna_{},{},RNA\n".format(s,cond))
            hdr.append("rna_{}".format(s))

    print(len(samples),len(hdr))
    o.write('\t'.join(hdr)+'\n')
    #o.write("\t".join(['ribo_{}'.format(s) for s in samples])+'\n')
    for g in gsub:
        o.write("{}\t{}\t{}\n".format(g,'\t'.join([str(int(rDict[rb][i][g])) for i in samples]),'\t'.join([str(int(rDict[rn][i][g])) for i in samples])))


riboPLATE_ERCC.final
riboPLATE_ERCC.final.cts.txt riboPLATE_ERCC.final.cols.txt
96 192


In [86]:
#Setup for DESeq2 on riboseq/RNAseq dataset: excluding ERCC
samples = [e for e in fullseq]
conditions = list(set(e[0][:-2] for e in samples))
print(samples)
print(conditions)
conditionDict = {c:[s for s in samples if s[0][:-2]==c] for c in conditions}
print(conditionDict)
conditionsTS = conditions

ctFl = 'FL.final.cts.txt'
clFl = 'FL.final.cols.txt'

ggs = [g for g in fullseq[samples[0]].keys() if g in goodG and 'ERCC-' not in g]
lbl = '.final'
with open(wDir+ctFl,'w') as o, open(wDir+clFl,'w') as col:
    col.write('condition,type\n')
    hdr = []
    samples = []
    for c in conditions:
        for s in conditionDict[c]:
            col.write('{},{},{}\n'.format('_'.join(s),c,s[1]))
            samples.append(s)
            hdr.append('_'.join(s))
    o.write('\t'.join(hdr)+'\n')
    for g in ggs:
        o.write("{}\t{}\n".format(g,'\t'.join([str(int(fullseq[s][g])) for s in samples])))


[('DMSO_30_1', 'RIBO'), ('DMSO_30_2', 'RIBO'), ('DMSO_30_3', 'RIBO'), ('PP_30_1', 'RIBO'), ('PP_30_2', 'RIBO'), ('PP_30_3', 'RIBO'), ('PP_6_1', 'RIBO'), ('PP_6_3', 'RIBO'), ('DMSO_6_3', 'RIBO'), ('DMSO_6_4', 'RIBO'), ('DMSO_30_1', 'RNA'), ('DMSO_30_2', 'RNA'), ('DMSO_30_3', 'RNA'), ('PP_30_1', 'RNA'), ('PP_30_2', 'RNA'), ('PP_30_3', 'RNA'), ('DMSO_6_1', 'RNA'), ('DMSO_6_2', 'RNA'), ('DMSO_6_3', 'RNA'), ('PP_6_1', 'RNA'), ('PP_6_2', 'RNA'), ('PP_6_3', 'RNA'), ('DMSO_6_4', 'RNA')]
['DMSO_6', 'DMSO_30', 'PP_30', 'PP_6']
{'DMSO_6': [('DMSO_6_3', 'RIBO'), ('DMSO_6_4', 'RIBO'), ('DMSO_6_1', 'RNA'), ('DMSO_6_2', 'RNA'), ('DMSO_6_3', 'RNA'), ('DMSO_6_4', 'RNA')], 'DMSO_30': [('DMSO_30_1', 'RIBO'), ('DMSO_30_2', 'RIBO'), ('DMSO_30_3', 'RIBO'), ('DMSO_30_1', 'RNA'), ('DMSO_30_2', 'RNA'), ('DMSO_30_3', 'RNA')], 'PP_30': [('PP_30_1', 'RIBO'), ('PP_30_2', 'RIBO'), ('PP_30_3', 'RIBO'), ('PP_30_1', 'RNA'), ('PP_30_2', 'RNA'), ('PP_30_3', 'RNA')], 'PP_6': [('PP_6_1', 'RIBO'), ('PP_6_3', 'RIBO'), ('PP_

In [116]:
cmd = "Rscript deseq_xform.R {} riboPLATE.final ctint ~condition+type+condition:type".format(wDir)
os.system(cmd)
cmd = "Rscript deseq_xform_2b.R {} FL.final ctint ~condition+type+condition:type".format(wDir)
os.system(cmd)
cmd = "Rscript deseq_xform.R {} riboPLATE_ERCC.final tbasic ~type".format(wDir)
os.system(cmd)

cmd = "Rscript deseq_xform.R {} riboPLATE.final tbasic ~type".format(wDir)
os.system(cmd)
cmd = "Rscript deseq_xform_2b.R {} FL.final tbasic ~type".format(wDir)
os.system(cmd)

SyntaxError: invalid syntax (2068712720.py, line 1)

In [114]:
#SATURATION ANALYSIS
#runs concurrently, takes a long time

tsSamp = [k for k in fullseq]
IntervalNumber = 50
detG = []
sampleR = np.linspace(1e5,1e7,num=IntervalNumber)#np.logspace(4,7,num=IntervalNumber)
#t_S: vector of all sample IDs
#t_S = [(k,rb) for rb in ['riboPLATE','PLATE','riboPLATE_ERCC','PLATE_ERCC'] for k in range(96)]+[(e[1],e[0]) for e in fullseq]
for sampleNumber in [int(e) for e in sampleR]:
    ftDict = {}
    tmp = {}
    #print(int(i))
    #dicts = [{g:v for (g,v) in rDict[rb][k].items() if g in goodG} for rb in ['riboPLATE','PLATE','riboPLATE_ERCC','PLATE_ERCC'] for k in rDict[rb]]+[{g:v for (g,v) in fullseq[k].items() if g in goodG} for k in fullseq]# if 'DMSO' in k[0] or 'PP' in k[0]]
    #print(len(dicts))
    with cf.ProcessPoolExecutor() as executor:
        for rb in ['riboPLATE','PLATE','riboPLATE_ERCC','PLATE_ERCC']:
            for k in rDict[rb]:
                ftDict[executor.submit(sampleCounts,rDict[rb][k],sampleNumber)] = (rb,k)
        for k in fullseq:
            ftDict[executor.submit(sampleCounts,fullseq[k],sampleNumber)] = k
        for ft in cf.as_completed(ftDict):
            res = ft.result()
            k = ftDict[ft]
            if res != None:
                tmp[k]=len([g for g in res if res[g]>0])
            else:
                tmp[k]=0
        detG.append(tmp)
        #dicts0 = {rb:{k:{g:v for (g,v) in rDict[rb][k].items() if g in goodG} for k in rDict[rb]} for rb in ['riboPLATE','PLATE','riboPLATE_ERCC','PLATE_ERCC']}# for k in rDict[rb]]
        #dictGen = it.chain((rDict[r][k] for r in ['riboPLATE','PLATE','riboPLATE_ERCC','PLATE_ERCC'] for k in rDict[r]),(fullseq[k] for k in fullseq))
        #1/0
        #sampleRB = [e for e in executor.map(sampleCounts,zip(dicts,[i for _ in range(len(dicts))]))]
        #sampleRB = executor.map(sampleCounts,dicts,(sampleNumber for  _ in range(len(dicts))))
        #tmp = {t_S[j]:len([g for g in srb if srb[g]>0]) if srb!=None else 0 for j,srb in enumerate(sampleRB)}
        #detG.append(tmp)        
        #tmp = {}
        #for j,srb in enumerate(sampleRB):
        #    if srb!=None:
        #        tmp[t_S[j]]= len([g for g in srb if srb[g]>0])
        #    else:
        #        tmp[t_S[j]] = 0
        #detG.append(tmp)
        
with open("{}SATURATION_OUT.txt".format(wDir),'w') as o:
    o.write('\t'.join([str(int(i)) for i in sampleR])+'\n')
    for rb in ['riboPLATE','PLATE','riboPLATE_ERCC','PLATE_ERCC']:
        for s in rDict[rb]:
            o.write("{}\t{}\n".format('.'.join([rb,s]),'\t'.join([str(detG[i][(rb,s)]) for i in range(len(sampleR))])))
    for k in fullseq:
        o.write("{}\t{}\n".format('.'.join(k),'\t'.join([str(detG[i][k]) for i in range(len(sampleR))])))        

Process ForkProcess-97:
Process ForkProcess-106:
Process ForkProcess-104:
Process ForkProcess-102:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/lib/python3.9/concurrent/futures/process.py", line 237, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/ubuntu/miniconda3/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/ubuntu/miniconda3/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._k

KeyboardInterrupt: 

In [ ]:
#LIBRARY SATURATION PLOTS
#first run 
satfile = '{}SATURATION_OUT.txt'.format(wDir)
satDict = {}
with open(satfile,'r') as f:
    hdr = [int(e) for e in f.readline().strip().split()]
    for line in f:
        e = line.split()
        satDict[e[0]] = {hdr[i]:int(e[i+1]) for i in range(len(hdr))}

rpf = [e for e in satDict if 'RIBO' in e]
rna = [e for e in satDict if 'RNA' in e]
riboplate = [e for e in satDict if 'riboPLATE_ERCC' in e]
rnaplate = [e for e in satDict if 'PLATE_ERCC' in e]
riboplate2 = [e for e in satDict if 'riboPLATE' in e and e not in riboplate]
rnaplate2 = [e for e in satDict if 'PLATE' in e and e not in rnaplate]

for sset,lbl in zip((rpf,rna,riboplate,rnaplate,riboplate2,rnaplate2),('Ribosome Profiling','RNA-seq','riboPLATE-seq_ERCC','PLATE-seq_ERCC','riboPLATE-seq','PLATE-seq')):
    pp = PdfPages("{}saturation_{}.pdf".format(outdir,'_'.join(lbl)))
    plt.figure(figsize=(10,10))
    print(len(sset),len(hdr))
    if lbl=='Ribosome Profiling':
        plt.xlim(0,5e6)
    elif lbl=='RNA-seq':
        plt.xlim(0,8e6)
    elif lbl == 'riboPLATE-seq':
        plt.xlim(0,1.2e6)
    elif lbl == 'PLATE-seq':
        plt.xlim(0,1e6)
        
    elif lbl == 'riboPLATE-seq_ERCC':
        plt.xlim(0,1.5e5)#200000)
    elif lbl == 'PLATE-seq_ERCC':
        plt.xlim(0,1e6)

    for e in sset:
        validR = [v for v in hdr if satDict[e][v]>0]
        #print(len(validR))
        if len(validR)==0:
            continue
        if 'ERCC' not in lbl:
            validR = [validR[i] for i in (0,4,7)]+validR[10:]
        else:
            validR = [validR[i] for i in [0]]+validR[2:]

        sns.scatterplot(x=[v for v in validR],
                        y=[satDict[e][v] for v in validR],
                        color='black',
                        edgecolor='none'
                       )
    plt.ylabel("Unique Genes Detected")
    plt.xlabel("Sampling Depth (Counts Sampled)")
    plt.title(lbl)
    pp.savefig()
    plt.show()
    pp.close()

In [ ]:
#Overall plots of genes vs counts, for all libraries (excluding outliers)
badS = [0,11,36,60,76]
pp = PdfPages("{}Complexity.pdf".format(outdir))
plt.figure(figsize=(9,9))
plt.title("All libraries")
for run in ['riboPLATE','riboseq']:    
    if run=='riboPLATE':
        rbDict = rDict[run]
        rnDict = rDict['PLATE']
        smps = range(96)
    else:
        rbDict = {k[0]:v for (k,v) in fullseq.items() if k[1]=='RIBO'}
        rnDict = {k[0]:v for (k,v) in fullseq.items() if k[1]=='RNA'}
    ssmp = [s for s in rbDict if s in rnDict]
    if run == 'riboPLATE':
        rbSum = {s:sum([rbDict[s][v] for v in rbDict[s] if v not in badG and 'ERCC-' not in v]) for s in ssmp}
        rnSum = {s:sum([rnDict[s][v] for v in rnDict[s] if v not in badG and 'ERCC-' not in v]) for s in ssmp}
        medRD = [np.mean([e for e in rbSum.values()]),np.mean([e for e in rnSum.values()])]
    gs = [e for e in rbDict[ssmp[0]].keys() if e not in badG and 'ERCC-' not in e]
    print(len(rbDict),len(rnDict),len(ssmp))
    plt.scatter([sum([rbDict[s][g] for g in gs]) for s in ssmp],[len([g for g in rbDict[s] if g in gs and rbDict[s][g]>1]) for s in ssmp],label='{} ribo'.format(run),alpha=0.6,edgecolor='none')
    plt.scatter([sum([rnDict[s][g] for g in gs]) for s in ssmp],[len([g for g in rnDict[s] if g in gs and rnDict[s][g]>1]) for s in ssmp],label='{} rna'.format(run),alpha=0.6,edgecolor='none')
rbDict = {k[0]:{g:v for (g,v) in fullseq[k].items() if g not in badG and 'ERCC-' not in g} for k in fullseq}
rnDict = {k[0]:{g:v for (g,v) in fullseq[k].items() if g not in badG and 'ERCC-' not in g} for k in fullseq}
rbds = {k:sampleCounts(rbDict[k],int(medRD[0])) for k in rbDict}
rnds = {k:sampleCounts(rnDict[k],int(medRD[1])) for k in rnDict}
plt.scatter([sum([rbds[s][g] for g in rbds[s]]) for s in ssmp],[len([g for g in rbds[s] if rbds[s][g]>1]) for s in ssmp],label='{} ribo downsampled'.format(run),alpha=0.6,edgecolor='none')
plt.scatter([sum([rnds[s][g] for g in rnds[s]]) for s in ssmp],[len([g for g in rnds[s] if rnds[s][g]>1]) for s in ssmp],label='{} rna downsampled'.format(run),alpha=0.6,edgecolor='none')

plt.xlabel("Sum counts")
plt.ylabel("Genes detected")
plt.gca().set_xscale('log')
plt.legend()
pp.savefig()
plt.show()
pp.close()

In [ ]:
#Load DESeq2 results
resDict = {}
sigGs = [set(),set()]
mod = 'ctint'
for c in ['PP242','MNK1','4EGI1','AZD','BKM','MNK1BKM','PP242BKM','PP242MNK1']:
    fl = '{od}riboPLATE.final.{md}.condition{c}.typeRIBO.{c}.results.txt'.format(od=wDir,md=mod,c=c)#oDir+'.final.'+mod+'condition{}'
    if not os.path.exists(fl):
        print(fl)
        print(c," NOT FOUND")
    else:
        with open(fl,'r') as f:
            allres = [l.split() for l in f.readlines()[1:]]
            resDict[c] = {l[0]:((float(l[2]),float(l[-1])) if 'NA'!=l[-1] else (float(l[2]),1.0)) for l in allres if l[2]!='NA'}    
        sigGs[0] |= set([g for g in resDict[c] if resDict[c][g][1]<0.05 and resDict[c][g][0]<0]) 
        sigGs[1] |= set([g for g in resDict[c] if resDict[c][g][1]<0.05 and resDict[c][g][0]>0]) 
print(len(sigGs[0]),len(sigGs[1]))

for cpfx in ['_6','30']:
    fl = '{od}FL.final.{md}.conditionPP{pf}.typeRIBO.DMSO{pf}.results.txt'.format(od=wDir,md=mod,pf=cpfx)
    if not os.path.exists(fl):
        print(fl)
        print('PP'+c," NOT FOUND")
    else:
        with open(fl,'r') as f:
            allres = [l.split() for l in f.readlines()[1:]]
            resDict['PP'+c] = {l[0]:((float(l[2]),float(l[-1])) if 'NA'!=l[-1] else (float(l[2]),1.0)) for l in allres if l[2]!='NA'}    


In [ ]:
#GSEA setup: genesets (on p-value and direction) AND rankedlists (on effect size)
tid='riboPLATE'
setDict = {}
setDict_excl = {}
rankDict = {}
cc = ['PP242','MNK1','4EGI1','BKM','AZD']#'MNK1BKM','PP242BKM','PP242MNK1']
combos = ['MNK1BKM','PP242BKM','PP242MNK1']
for r in cc+combos:
    setDict[r] = ([g for g in resDict[r] if resDict[r][g][1]<0.05 and resDict[r][g][0]<0],[g for g in resDict[r] if resDict[r][g][1]<0.05 and resDict[r][g][0]>0])
    rankDict[r] = {g:resDict[r][g][0] for g in resDict[r]}
    print(r,len(setDict[r][0]),len(setDict[r][1]))
#exclusive sets are only exclusive w/in single- and double-treated sets
for r in cc:
    setDict_excl[r] = ([g for g in setDict[r][0] if len([_ for r in cc if g in setDict[r][0]])==1],[g for g in setDict[r][1] if len([_ for r in cc if g in setDict[r][1]])==1])
    print(r,len(setDict_excl[r][0]),len(setDict_excl[r][1]))
for r in combos:
    setDict_excl[r] = ([g for g in setDict[r][0] if len([_ for r in combos if g in setDict[r][0]])==1],[g for g in setDict[r][1] if len([_ for r in combos if g in setDict[r][1]])==1])
    print(r,len(setDict_excl[r][0]),len(setDict_excl[r][1]))

setsAcrossCombos = [(set(setDict[c][0]),set(setDict[c][1])) for c in combos]
setsInit = setsAcrossCombos[0]
for c in range(len(combos)):
    setsInit = (setsInit[0]&setsAcrossCombos[c][0],setsInit[1]&setsAcrossCombos[c][1])
    print('COMBOS',len(setsInit[0]),len(setsInit[1]))
setsAcrossCombosE = [(set(setDict[c][0]),set(setDict[c][1])) for c in combos]
setsInitAll = setsAcrossCombosE[0]
for c in range(len(combos)):
    setsInitAll = (setsInitAll[0]|setsAcrossCombosE[c][0],setsInitAll[1]|setsAcrossCombosE[c][1])
    print('INCL COMBOS',len(setsInitAll[0]),len(setsInitAll[1]))

setDict_b = {}
rankDict_b = {}
for r in ['PP_30','PP_6']:
    setDict_b[r] = ([g for g in resDict[r] if resDict[r][g][1]<0.05 and resDict[r][g][0]<0],[g for g in resDict[r] if resDict[r][g][1]<0.05 and resDict[r][g][0]>0])
    rankDict_b[r] = {g:resDict[r][g][0] for g in resDict[r]}
    print(r,len(setDict_b[r][0]),len(setDict_b[r][1]))
rankDict.update(rankDict_b)


setDict_bexcl = {}
for r in setDict_b:
    setDict_bexcl[r] = ([g for g in setDict_b[r][0] if len([_ for r in setDict_b if g in setDict_b[r][0]])==1],[g for g in setDict_b[r][1] if len([_ for r in setDict_b if g in setDict_b[r][1]])==1])
    print(r,len(setDict_bexcl[r][0]),len(setDict_bexcl[r][1]))
    
gmtfile = wDir+'{}.gmt'.format(tid)
with open(gmtfile,'w') as o:
    for c in cc:
        if len(setDict[c][0]):
            o.write("{}_DN\tna\t{}\n".format(c,'\t'.join(setDict[c][0])))
        if len(setDict[c][1]):
            o.write("{}_UP\tna\t{}\n".format(c,'\t'.join(setDict[c][1])))
    for c in ['PP_30','PP_6']:
        if len(setDict_b[c][0]):
            o.write("{}_DN\tna\t{}\n".format(c,'\t'.join(setDict_b[c][0])))
        if len(setDict_b[c][1]):
            o.write("{}_UP\tna\t{}\n".format(c,'\t'.join(setDict_b[c][1])))

gmtfile = wDir+'{}_combos.gmt'.format(tid)
with open(gmtfile,'w') as o:
    for c in combos:
        if len(setDict[c][0]):
            o.write("{}_DN\tna\t{}\n".format(c,'\t'.join(setDict[c][0])))
        if len(setDict[c][1]):
            o.write("{}_UP\tna\t{}\n".format(c,'\t'.join(setDict[c][1])))        
    o.write("COMBO_DN\tna\t{}\n".format('\t'.join(setsInitAll[0])))      
    o.write("COMBO_UP\tna\t{}\n".format('\t'.join(setsInitAll[1])))      
            
gmtfile = wDir+'{}_excl.gmt'.format(tid)
with open(gmtfile,'w') as o:
    for c in cc:
        if len(setDict_excl[c][0]):
            o.write("{}excl_DN\tna\t{}\n".format(c,'\t'.join(setDict_excl[c][0])))
        if len(setDict_excl[c][1]):
            o.write("{}excl_UP\tna\t{}\n".format(c,'\t'.join(setDict_excl[c][1])))
    for c in ['PP_30','PP_6']:
        if len(setDict_bexcl[c][0]):
            o.write("{}excl_DN\tna\t{}\n".format(c,'\t'.join(setDict_bexcl[c][0])))
        if len(setDict_bexcl[c][1]):
            o.write("{}excl_UP\tna\t{}\n".format(c,'\t'.join(setDict_bexcl[c][1])))

for r in rankDict:
    with open(wDir+"{}.rnk".format(r),'w') as o:
        for g in [g for g in ggs if g in rankDict[r]]:
            o.write('{}\t{:.2f}\n'.format(g,round(rankDict[r][g],2)+0))
            
cmd = "batchGSEA.py {} {} {}".format(wDir,gsdir,gspath)
print(cmd)
os.system(cmd)

In [ ]:
pp = PdfPages("F3_sigGenesBar.pdf")
rsd = [r for r in resDict if r not in ('PP_30','PP_6')]
plt.bar(range(len(rsd)),[len([g for g in resDict[r] if resDict[r][g][-1]<0.05 and resDict[r][g][0]<0]) for r in rsd],color='blue',label='down')
plt.bar(range(len(rsd)),[len([g for g in resDict[r] if resDict[r][g][-1]<0.05 and resDict[r][g][0]>0]) for r in rsd],color='red',label='up',bottom=[len([g for g in resDict[r] if resDict[r][g][-1]<0.05 and resDict[r][g][0]<0]) for r in rsd])
plt.xticks(range(len(rsd)),rsd,rotation='vertical')
plt.legend()
plt.title("Significant genes by condition")
plt.ylabel("Number of significant changes in RA (FDR<0.05)")
plt.xlabel("Drug tested")
pp.savefig()
#plt.xlim(-0.5,7.5)
plt.show()
                                                                                                                                       
pp.close()

In [ ]:
#TOP gene tests
for r in resDict:
    print(r,stats.wilcoxon([resDict[r][g][0] for g in resDict[r] if resDict[r][g][-1]<0.05 and g in classicTOP],correction=True,alternative='less'))
    print(r,stats.wilcoxon([resDict[r][g][0] for g in resDict[r] if g in classicTOP],correction=True,alternative='less'))
    print(r,stats.mannwhitneyu([resDict[r][g][0] for g in resDict[r] if g in classicTOP],[resDict[r][g][0] for g in resDict[r] if g not in classicTOP],alternative='less'))
    print(r,stats.mannwhitneyu([resDict[r][g][0] for g in resDict[r] if g in classicTOP and resDict[r][g][-1]<0.05],[resDict[r][g][0] for g in resDict[r] if g not in classicTOP and resDict[r][g][-1]<0.05],alternative='less'))
    
    s_T = len([g for g in resDict[r] if resDict[r][g][-1]<0.05 and resDict[r][g][0]<0 and g in classicTOP])
    s_nT =len([g for g in resDict[r] if resDict[r][g][-1]<0.05 and resDict[r][g][0]<0  and g not in classicTOP])
    ns_T = len([g for g in resDict[r] if resDict[r][g][-1]>=0.05 and resDict[r][g][0]<0  and g in classicTOP])
    ns_nT = len([g for g in resDict[r] if resDict[r][g][-1]>=0.5 and resDict[r][g][0]<0  and g not in classicTOP])
    print(s_T,s_nT,ns_T,ns_nT)
    print(stats.fisher_exact([[s_T,ns_T],[s_nT,ns_nT]]))

In [ ]:
##GENERATE PCA ON ERCC-FREE MATRICES (w/o normalization wrt ERCC)
#PC1 correlation vs effect magnitude/significance
#TODO: make sure PD022024 PCAs use appropriate (1) outlier lists and (2) underlying genes (i.e. significant in DESeq2)
ccmap = plt.get_cmap('Set1')
#genes = [g for g in rDict['PN024_ribo'][0].keys() if g not in badG]
#erccs = [g for g in genes if 'ERCC-' in g]
ddir = wDir
pfx = 'riboPLATE.final'
for mod in ['ctint','tbasic']:
    for norm in ['vst','norm']:
        try:
                fl = ddir+'{p}.{md}.{norm}.txt'.format(p=pfx,md=mod,norm=norm)
                allcts = readMatrix(fl)
                ctDict = {s:{g:float(v) for (g,v) in allcts[s].items()} for s in allcts}
                riboD = {s.split('_')[1]:v for (s,v) in ctDict.items() if 'ribo' in s}                    
                rnaD = {s.split('_')[1]:v for (s,v) in ctDict.items() if 'rna' in s}
                sampsA = sorted(list(set([e for e in riboD.keys()])|set([e for e in rnaD.keys()])))
                print(len(sampsA))
                teD = {k:{v:float(riboD[k][v])-float(rnaD[k][v]) for v in riboD[k] if v in rnaD[k]} for k in riboD if k in rnaD}                
                ggenes = [g for g in teD[sampsA[0]].keys() if g in sigGs[0]|sigGs[1]]
                print("{} genes".format(len(ggenes)))
                lbl = ' '.join([pfx,norm,mod])
                pp = PdfPages("{}{}_PCA.pdf".format(outdir,'.'.join(mod,norm)))
                condsA = [maplist[int(i)] for i in sampsA]
                groupsA = list(set(condsA))
                grpsA = {i:e for (i,e) in enumerate(condsA)}                    
                conditionIdx = {c:i for i,c in enumerate(groupsA)}
                NUM_COLORS = len(groupsA)
                colors = {grp:ccmap(conditionIdx[grp]/NUM_COLORS) for grp in groupsA}
                dIdx = ['ribo','rna','RA']
                for (idx,typD) in [(0,riboD),(1,rnaD),(2,teD)]:
                    tmp = [(s,condsA[i]) for (i,s) in enumerate(sampsA) if s in typD] 
                    samps = [s[0] for s in tmp]
                    conds = [s[1] for s in tmp]
                    dat = [[typD[s][g] for g in ggenes] for s in samps]
                    plt.figure(figsize=(5,5))
                    plt.title(dIdx[idx])
                    #Change False to True to put sample annotations on each point in the PCA
                    plotPCA(plt.gca(),dat,samps,conds,colors,False)
                    pp.savefig()
                    plt.show()
                ctlS = [k for k in teD if maplist[int(k)]=='CTRL']
                meanCTL = {g:np.mean([teD[k][g] for k in teD if maplist[int(k)]=='CTRL']) for g in teD[sampsA[0]].keys()}
                lfcTE = {k:{g:teD[k][g]-meanCTL[g] for g in teD[k]} for k in teD if maplist[int(k)]!='CTRL'}
                ggenes = [g for g in teD[sampsA[0]].keys()]
                print("{} genes".format(len(ggenes)))
                plt.figure(figsize=(5,5))
                sampsE = sorted(list([s for s in lfcTE.keys()]))
                cndsE = [maplist[int(e)] for e in sampsE]
                dat = [[lfcTE[s][g] for g in ggenes] for s in sampsE]
                plotPCA(plt.gca(),dat,sampsE,cndsE,colors,False)
                plt.title("log-fold change in RA vs DMSO")
                pp.savefig()
                plt.show()
                
                plt.figure()
                handles = [mpatches.Patch(color=colors[cond],label=cond) for cond in colors]
                plt.legend(handles=handles,frameon=False)
                plt.gca().set_axis_off()
                pp.savefig()
                plt.show()
                pp.close()
                
                meanPC = {c:(np.mean([res[i][0] for i,s in enumerate(sampsE) if maplist[int(s)]==c]),
                             np.mean([res[i][1] for i,s in enumerate(sampsE) if maplist[int(s)]==c]),                              
                             np.mean([np.mean([np.abs(lfcTE[s][g]) for s in lfcTE if maplist[int(s)]==c]) for g in sigGs[0]|sigGs[1] if g in resDict[''.join(c.split('+'))]]),
                             len([g for g in resDict[''.join(c.split('+'))] if resDict[''.join(c.split('+'))][g][1]<0.05]),
                            )
                          for c in set(maplist) if c!='CTRL'}
                indPC = {s:(res[i][0],res[i][1],
                            np.mean([np.abs(lfcTE[s][g]) for g in sigGs[0]|sigGs[1]])
                           ) for (i,s) in enumerate(sampsE)}
                plt.scatter([meanPC[c][0] for c in meanPC],[meanPC[c][3] for c in meanPC],c=[colors[c] for c in meanPC])
                plt.xlabel("PC1")
                plt.ylabel("Number of significant effects (across condition)")
                plt.show()
                print(stats.spearmanr([meanPC[c][0] for c in meanPC],[meanPC[c][3] for c in meanPC]))
                plt.scatter([indPC[s][0] for s in indPC],[indPC[s][2] for s in indPC],c=[colors[maplist[int(s)]] for s in indPC])
                plt.xlabel("PC1")
                plt.ylabel("mean absolute effect (individual samples)")
                plt.show()                
                print(stats.spearmanr([indPC[s][0] for s in indPC],[indPC[s][2] for s in indPC]))                       

            except OSError:
                print(fl)


In [ ]:
#Depletion scatterplots -- use tbasic vst for sake of comparison (or norm, since it's closely related to depletion histograms w norm counts)
for mod in ['tbasic','ctint']:
    for norm in ['norm','vst']:
        for pfx in ['FL','riboPLATE','riboPLATE_ERCC']:#[::-1]:                
            try:
                fl = wDir+'{p}.final.{md}.{norm}.txt'.format(p=pfx,md=mod,norm=norm)
                allcts = readMatrix(fl)
            except Exception:
                print (mod,norm,pfx,"NOT FOUND")
                continue
                
            ctDict = {s:{g:float(v) for (g,v) in allcts[s].items()} for s in allcts}
            if 'FL' == pfx:
                riboD = {'_'.join(s.split('_')[:-1]):v for (s,v) in ctDict.items() if 'RIBO' in s}
                rnaD = {'_'.join(s.split('_')[:-1]):v for (s,v) in ctDict.items() if 'RNA' in s}
            else:
                riboD = {s.split('_')[1]:v for (s,v) in ctDict.items() if 'ribo' in s}                    
                rnaD = {s.split('_')[1]:v for (s,v) in ctDict.items() if 'rna' in s}
            sampsA = sorted(list(set([e for e in riboD.keys()])|set([e for e in rnaD.keys()])))
            print(len(sampsA))
            teD = {k:{v:float(riboD[k][v])-float(rnaD[k][v]) for v in riboD[k] if v in rnaD[k]} for k in riboD if k in rnaD}                
            ggenes = [g for g in teD[sampsA[0]].keys()]# if g in sigG[0]+sigG[1]]                
            #ncRNA scatterplots
            lbl='.'.join((pfx,norm,mod))
            pp = PdfPages("{}{}_depletion_scatter.pdf".format(outdir,lbl))                    
            dict1 = rnaD
            dict2 = teD
            samps = [s for s in dict1 if s in dict2]
            print(len(samps))
            plt.figure(figsize=(5,5))
            plt.title('_'.join([pfx,mod,norm]))
            if pfx=='riboPLATE_ERCC':
                smpE = [str(e) for e in range(96)[::2] if str(e) in sampsA]
            else:
                smpE = samps
            plt.scatter([np.mean([rnaD[s][g] for s in smpE]) for g in ggenes if g not in ncGenes and 'ERCC-' not in g],[np.mean([teD[s][g] for s in smpE]) for g in ggenes if g not in ncGenes and 'ERCC-' not in g],label='mRNA',color='blue',alpha=0.5,s=20,edgecolors='none')
            plt.scatter([np.mean([rnaD[s][g] for s in smpE]) for g in ggenes if g in ncGenes],[np.mean([teD[s][g] for s in smpE]) for g in ggenes if g in ncGenes],label='ncRNA',color='red',alpha=0.5,s=20,edgecolors='none')
            plt.legend()
            print(stats.wilcoxon([np.mean([teD[s][g]/rnaD[s][g] for g in ggenes if g in ncGenes and 'ERCC-' not in g]) for s in smpE], [np.mean([teD[s][g]/rnaD[s][g] for g in ggenes if g not in ncGenes and 'ERCC-' not in g]) for s in smpE],correction=True,alternative='greater'))
            plt.xlabel('RNA')#l.split('V')[0])
            plt.ylabel('RA')
            pp.savefig()
            plt.show()
            if pfx == 'PN024':
                plt.figure(figsize=(5,5))
                plt.title('_'.join([pfx,mod,norm]))
                ercG = [g for g in ggenes if 'ERCC-' in g]
                nrcG = [g for g in ggenes if 'ERCC-' not in g and g not in ncGenes]
                xs = [np.mean([rnaD[s][g] for s in smpE]) for g in nrcG]
                ys = [np.mean([teD[s][g] for s in smpE]) for g in nrcG]
                plt.scatter(xs,ys,label='mRNA',color='blue',alpha=0.5,s=20,edgecolors='none')
                xs = [np.mean([rnaD[s][g] for s in smpE]) for g in ercG]
                ys = [np.mean([teD[s][g] for s in smpE]) for g in ercG]
                plt.scatter(xs,ys,label='ERCC',color='red',alpha=0.5,s=20,edgecolors='none')
                plt.xlabel("RNA")
                plt.ylabel("RA")
                print(stats.wilcoxon([np.mean([teD[s][g] for g in nrcG]) for s in smpE], [np.mean([teD[s][g] for g in ercG]) for s in smpE],correction=True,alternative='greater'))
                plt.legend()
                pp.savefig()
                plt.show()
            pp.close()


In [ ]:
for c in ['PP242','AZD','BKM','MNK1','4EGI1']:#,'PP_6','PP_30']:
    pp = PdfPages("{}{}_volcano.pdf".format(outdir,c))
    plt.figure(figsize=(6,6))
    ymax=10
    if c!='4EGI1':
        xm = 2.5
    else:
        xm = 7.9
    #In loading resDict, p values of NA converted to 1 --> many changes w/ no significance computed b/c outliers or overdispersion
    #Must be filtered for the sake of displaying volcano plot, but what about in ranked lists and other comparisons across conditions?
    #If idea is to demonstrate effect sizes w/o regard to significance, leave them in
    #If that's not a realistic application of the technology and significance should be restricted, how do things change downstream?
    
    plt.scatter([resDict[c][g][0] for g in resDict[c] if g not in classicTOP and resDict[c][g][1]!=1.0],[-np.log10(resDict[c][g][1]) if resDict[c][g][1]!=0 and -np.log10(resDict[c][g][1])<ymax else ymax for g in resDict[c] if g not in classicTOP and resDict[c][g][1]!=1.0],alpha=0.7,s=30,label='non-TOP',color='black',edgecolors='none')
    #plt.scatter([resDict[c][g][0] for g in genes if g in allTOP],[-np.log10(resDict[c][g][1]+1e-40) for g in genes if g in allTOP],alpha=0.5,s=3,label='TOP',color='red')
    plt.scatter([resDict[c][g][0] for g in resDict[c] if g in classicTOP and resDict[c][g][1]!=1.0],[-np.log10(resDict[c][g][1]) if resDict[c][g][1]!=0 and -np.log10(resDict[c][g][1])<ymax else ymax for g in resDict[c] if g in classicTOP and resDict[c][g][1]!=1.0],alpha=0.85,s=30,label='TOP',color='green',edgecolors='none')#n([resDict[c][g][1] for g in resDict[c] if resDict[c][g][1]])/10
    plt.axhline(-np.log10(0.05),linestyle='dashed',color='teal',alpha=0.5)
    plt.title(c)
    #plt.xlim(-5,5)
    plt.ylim(0,ymax+0.2)
    plt.xlim(-xm,xm)
    plt.axvline(1,linestyle='dashed',alpha=0.5,color='teal')
    plt.axvline(-1,linestyle='dashed',alpha=0.5,color='teal')
    plt.title(c)
    plt.xlabel("Log-Fold Change in RA, vs DMSO")
    plt.ylabel("Significance (-log(FDR))")
    plt.legend()
    pp.savefig()
    plt.show()
    print(stats.mannwhitneyu([resDict[c][g][0] for g in resDict[c] if g in classicTOP],[resDict[c][g][0] for g in resDict[c] if g not in classicTOP],alternative='less',use_continuity=True))    
    pp.close()


In [ ]:
for c in ['PP_6','PP_30']:
    pp = PdfPages("{}{}_volcano.pdf".format(outdir,c))
    plt.figure(figsize=(6,6))
    ymax=10
    #MAKE TOP GENES GREEN, RIBOPLATE TRIANGLES CYAN/MAGENTA, AND BG COLOR BLUE
    #In loading resDict, p values of NA converted to 1 --> many changes w/ no significance computed b/c outliers or overdispersion
    #Must be filtered for the sake of displaying volcano plot, but what about in ranked lists and other comparisons across conditions?
    #If idea is to demonstrate effect sizes w/o regard to significance, leave them in
    #If that's not a realistic application of the technology and significance should be restricted, how do things change downstream?
    ppTarg = [g for g in resDict['PP242'] if resDict['PP242'][g][-1]<0.05 and resDict['PP242'][g][0]<0],[g for g in resDict['PP242'] if resDict['PP242'][g][-1]<0.05 and resDict['PP242'][g][0]>0]
    #scatter background genome -- not TOP, not target of PP242 in riboPLATE
    xs = [resDict[c][g][0] for g in resDict[c] if g not in classicTOP and resDict[c][g][1]!=1.0 and g not in ppTarg[0]+ppTarg[1]]
    ys = [-np.log10(resDict[c][g][1]) if resDict[c][g][1]!=0 and -np.log10(resDict[c][g][1])<ymax else ymax for g in resDict[c] if g not in classicTOP and resDict[c][g][1]!=1.0 and g not in ppTarg[0]+ppTarg[1]]
    plt.scatter(xs,ys,color='black',label='background',s=20,alpha=0.3,edgecolors='none')
    xs = [resDict[c][g][0] for g in resDict[c] if g in ppTarg[1]]
    ys = [-np.log10(resDict[c][g][1]) if resDict[c][g][1]!=0 and -np.log10(resDict[c][g][1])<ymax else ymax for g in resDict[c] if g in ppTarg[1]]
    plt.scatter(xs,ys,color='red',alpha=0.7,s=30,edgecolors='none',label='riboPLATE up')
    xs = [resDict[c][g][0] for g in resDict[c] if g in ppTarg[0]]
    ys = [-np.log10(resDict[c][g][1]) if resDict[c][g][1]!=0 and -np.log10(resDict[c][g][1])<ymax else ymax for g in resDict[c] if g in ppTarg[0]]
    plt.scatter(xs,ys,color='blue',alpha=0.7,s=30,edgecolors='none',label='riboPLATE down')

    xs = [resDict[c][g][0] for g in resDict[c] if g in classicTOP and resDict[c][g][1]!=1.0]# and g not in ppTarg[0]+ppTarg[1]]
    ys = [-np.log10(resDict[c][g][1]) if resDict[c][g][1]!=0 and -np.log10(resDict[c][g][1])<ymax else ymax for g in resDict[c] if g in classicTOP and resDict[c][g][1]!=1.0]# and resDict[c][g][1]!=1.0 and g not in ppTarg[0]+ppTarg[1]]    
    plt.scatter(xs,ys,color='green',label='TOP',alpha=0.5,s=30,edgecolors='None')
    
    plt.axhline(y=-np.log10(0.05),linestyle='dashed',color='teal',alpha=0.5)
    plt.axvline(x=-1,linestyle='dashed',color='teal',alpha=0.5)
    plt.axvline(x=1,linestyle='dashed',color='teal',alpha=0.5)
    xmax = max([np.abs(resDict[c][g][0]) for g in resDict[c]])
    if c=='PP_6':
        plt.xlim(-7,7)
    elif c=='PP_30':
        plt.xlim(-11,11)
    plt.ylim(0,ymax+0.2)
    plt.title(c)
    plt.xlabel("log-fold change in TE, vs DMSO")
    plt.ylabel("Significance (-log(FDR))")
    plt.legend()
    pp.savefig()
    plt.show()
    print(stats.mannwhitneyu([resDict[c][g][0] for g in resDict[c] if g in classicTOP],[resDict[c][g][0] for g in resDict[c] if g not in classicTOP],use_continuity=True,alternative='less'))    
    print(stats.mannwhitneyu([resDict[c][g][0] for g in resDict[c] if g in ppTarg[0]],[resDict[c][g][0] for g in resDict[c] if g in ppTarg[1]],use_continuity=True,alternative='less'))    

    pp.close()

In [ ]:
#GSEA 1: heatmap of plate-riboseq effect correlation
gsdir = gsDir+'out/'
gsDict = {}
for rnk in ['PP242','MNK1','AZD','BKM','4EGI1','PP_6','PP_30']:#['PP_6','PP_30']:
    dfl = [e for e in os.listdir(gsdir) if rnk in e.split('.')[0] and 'riboPLATE' in e.split('.')[0] and 'combos' not in e and 'excl' in e]
    dfl = dfl[0]
    print(dfl)
    dpos = [e for e in os.listdir(gsdir+dfl) if 'gsea_report_for_na_pos' in e and '.tsv' in e]
    dneg = [e for e in os.listdir(gsdir+dfl) if 'gsea_report_for_na_neg' in e and '.tsv' in e]
    if len(dpos)>1 or len(dneg)>1:
        print("PROBLEM")
        print(dpos)
        print(dneg)
    dpos = dpos[0]
    dneg = dneg[0]
    with open(gsdir+dfl+'/'+dpos,'r') as fp, open(gsdir+dfl+'/'+dneg,'r') as fn:
        fp.readline()
        fn.readline()
        gsDict[rnk] = {e.split('\t')[0]:(e.split('\t')[5:9]) for e in fp.readlines()}
        gsDict[rnk].update({e.split('\t')[0]:(e.split('\t')[5:9]) for e in fn.readlines()})
sets = sorted([s for s in gsDict[rnk] if '_30' not in s and '_6' not in s])
sets = [s for s in sets if '_DN' in s][::-1]+[s for s in sets if '_UP' in s][::-1]
rnks = ['PP_6','PP_30']#,'4EGI1','MNK1','AZD','BKM','PP242']
#dat = [[float(gsDict[r][s][0]) for r in rnks if '_30' in r or '_6'  in r] for s in sets if '_30' not in s and '_6' not in s]# for r in rnks]
dat = [[float(gsDict[r][s][0]) for r in rnks] for s in sets]# for r in rnks]
nsig = [["*" if float(gsDict[r][s][-1])<0.05 else "" for r in rnks] for s in sets]

#dat = [[plateRes[c][g][0] for c in condes] for g in gset]
#rowcol = ['g' if g in allTOP else 'black' for g in gset]
metr = 'correlation'
#for metr in ['correlation','euclidean','seuclidean']:
pp = PdfPages(outdir+"riboPLATErpfGSEA.pdf")
plt.figure(figsize=(4,6))
sw = sns.heatmap(dat,xticklabels=rnks,annot=nsig,fmt='',yticklabels=sets,cmap='bwr',center=0,cbar_kws={'label':'GSEA Normalized Enrichment Score (NES)'})#,square=True)#,figsize=(5,10))
#    plt.title("{} distance".format(metr))
sw.set_xlabel("Drug Tested (log-fold change in TE vs DMSO, ribosome profiling/RNA-seq)")
sw.set_ylabel("Gene Sets of Drug Targets (FDR<0.05, riboPLATE/PLATE-seq RA)")
pp.savefig()
plt.show()
pp.close()

In [ ]:
ymax = 10
colors = it.cycle(('blue','red','cyan','magenta'))
for combo in [('PP242','BKM'),('PP242','MNK1'),('MNK1','BKM')]:
    cset = ''.join(combo)
    pp = PdfPages(outdir+"{}_combo_volcano.pdf".format(cset))
    plt.figure(figsize=(6,6))
    gs = set()
    for c in combo:
        gs_dn = [g for g in resDict[c] if resDict[c][g][-1]<0.05 and resDict[c][g][0]<0]#setDict_b[cset][0]
        gs_up = [g for g in resDict[c] if resDict[c][g][-1]<0.05 and resDict[c][g][0]>0]#setDict_b[cset][1]
        gs |= set(gs_dn)
        gs |= set(gs_up)
    plt.scatter([resDict[cset][g][0] for g in resDict[cset] if g not in gs and resDict[cset][g][-1]!=1],[-np.log10(resDict[cset][g][-1]) if resDict[cset][g][-1]!=0 and -np.log10(resDict[cset][g][-1])<ymax else ymax for g in resDict[cset] if g not in gs and resDict[cset][g][-1]!=1],color='black',alpha=0.5,label='bg',edgecolors='none')
    for c in combo:
        gs_dn = [g for g in resDict[c] if resDict[c][g][-1]<0.05 and resDict[c][g][0]<0]#setDict_b[cset][0]
        gs_up = [g for g in resDict[c] if resDict[c][g][-1]<0.05 and resDict[c][g][0]>0]#setDict_b[cset][1]    
        plt.scatter([resDict[cset][g][0] for g in resDict[cset] if g in gs_dn and resDict[cset][g][-1]!=1],[-np.log10(resDict[cset][g][-1]) if resDict[cset][g][-1]!=0 and -np.log10(resDict[cset][g][-1])<ymax else ymax for g in resDict[cset] if g in gs_dn and resDict[cset][g][-1]!=1],color=next(colors),alpha=0.3,edgecolors='none',label='{}_dn'.format(c))
        plt.scatter([resDict[cset][g][0] for g in resDict[cset] if g in gs_up and resDict[cset][g][-1]!=1],[-np.log10(resDict[cset][g][-1]) if resDict[cset][g][-1]!=0 and -np.log10(resDict[cset][g][-1])<ymax else ymax for g in resDict[cset] if g in gs_up and resDict[cset][g][-1]!=1],color=next(colors),alpha=0.3,edgecolors='none',label='{}_up'.format(c))
    plt.title("{}".format(cset))
    plt.ylim(0,ymax+0.2)
    plt.xlim(-5,5)
    plt.axhline(-np.log10(0.05),linestyle='dashed')
    plt.axvline(-1,alpha=0.5)
    plt.axvline(1,alpha=0.5)
    plt.legend()
    pp.savefig()
    plt.show()
    pp.close()

In [ ]:
cc = ['PP242','MNK1','4EGI1','BKM','AZD']#'MNK1BKM','PP242BKM','PP242MNK1']
combos = [('MNK1','BKM'),('PP242','BKM'),('PP242','MNK1')]
#Easier plot to digest: for each drug combo, plot effect size in singular vs effect size in combo for effects of both
#individual drugs
ccmap = plt.get_cmap('Set1')
grpsA = {i:e for (i,e) in enumerate(set(maplist))}                    
conditionIdx = {c:i for i,c in enumerate(set(maplist))}
NUM_COLORS = len(grpsA)
colors = {c:ccmap(conditionIdx[c]/NUM_COLORS) for c in conditionIdx}
for comb in combos:
    c = ''.join(comb)
    pp = PdfPages(outdir+"{}_combo_attenuation.pdf".format(c))    
    #gs: dict of gene:(effect,condition) for max effect seen in either individual condition
    gs = {}
    for ci in comb:
        for g in [g for g in resDict[ci] if resDict[ci][g][-1]<0.05]:
            try:
                if np.abs(resDict[ci][g][0])>np.abs(gs[g][0]):
                    gs[g] = (resDict[ci][g][0],ci)
                #gs[g] = np.max(np.abs(gs[g]),np.abs(resDict[ci][g][0]))
            except Exception:
                gs[g] = (resDict[ci][g][0],ci)
        #gs = {g:resDict[ci][g][0] for g in resDict[ci] if resDict[ci][g][-1]<0.05}
        #for g in gs:
        #    resDict[ci][g][-1]
        #gs  |= set([g for g in resDict[ci] if resDict[ci][g][-1]<0.05])
    #domX: set of genes significant in combination treatment
    domX = [g for g in resDict[c] if resDict[c][g][-1]<0.05]
    #ranX: set of genes significant in either individual treatment
    ranX = [g for g in gs]
    #xs = [resDict[c][g][0] for g in ranX]
    plt.figure(figsize=(5,5))

    #xs: effect size in individual drug of max effect
    xs = [gs[g][0] for g in ranX if g in resDict[c]]
    #ys: effect size in combination treatment
    ys = [resDict[c][g][0] for g in ranX if g in resDict[c]]
    #colors: condition in which max absolute effect is seen for target
    cs = [colors[gs[g][1]] for g in ranX if g in resDict[c]]
    plt.scatter(xs,ys,c=cs,alpha=0.75,edgecolors='none')
    plt.xlim(-3,3)
    plt.ylim(-3,3)
    #plt.ylim(-3.5,3.5)
    plt.axhline(0,linestyle='dashed',color='black')
    handles = [mpatches.Patch(color=colors[cond],label=cond) for cond in comb]
    plt.legend(handles=handles)
    #plt.axvline(0)
    plt.plot(range(-4,4),range(-4,4),linestyle='dashed',color='black')
    plt.title(c)
    plt.xlabel("Maximum effect observed in singular drug {}".format(comb))
    plt.ylabel("Observed effect in combination treatment")
    pp.savefig()
    plt.show()
    pp.close()
    #stdev of single drugs' upregulated/downregulated targets in combination treatment effect 
    print("Stdev up ",np.std([resDict[c][g][0] for g in gs if g in resDict[c] and resDict[gs[g][1]][g][0]>0]))
    print("Stdev dn ",np.std([resDict[c][g][0] for g in gs if g in resDict[c] and resDict[gs[g][1]][g][0]<0]))

    print(len([g for g in gs if np.sign(resDict[c][g][0])!=np.sign(resDict[gs[g][1]][g][0])])/len(gs),
         len([g for g in gs if np.sign(resDict[c][g][0])!=np.sign(resDict[gs[g][1]][g][0]) and resDict[gs[g][1]][g][0]>0])/len([g for g in gs if resDict[gs[g][1]][g][0]>0]),
         len([g for g in gs if np.sign(resDict[c][g][0])!=np.sign(resDict[gs[g][1]][g][0]) and resDict[gs[g][1]][g][0]<0])/len([g for g in gs if resDict[gs[g][1]][g][0]<0]),
         )
    print("percent attenuated overall ",len([g for g in gs if np.abs(resDict[c][g][0])<np.abs(resDict[gs[g][1]][g][0])])/len(gs),
          "percent attenuated up ",len([g for g in gs if np.abs(resDict[c][g][0])<np.abs(resDict[gs[g][1]][g][0]) and resDict[gs[g][1]][g][0]>0])/len([g for g in gs if resDict[gs[g][1]][g][0]>0]),
          "percent attenuated down ",len([g for g in gs if np.abs(resDict[c][g][0])<np.abs(resDict[gs[g][1]][g][0]) and resDict[gs[g][1]][g][0]<0])/len([g for g in gs if resDict[gs[g][1]][g][0]<0])
         )

    print("mean attenuation down ", np.mean([resDict[gs[g][1]][g][0]-resDict[c][g][0] for g in gs if resDict[gs[g][1]][g][0]<0 and resDict[c][g][0]>resDict[gs[g][1]][g][0]]),#/len([g for g in gs if resDict[gs[g][1]][g][0]<0]),
          "mean attenuation up ",np.mean([resDict[gs[g][1]][g][0]-resDict[c][g][0]  for g in gs if resDict[gs[g][1]][g][0]>0 and resDict[c][g][0]<resDict[gs[g][1]][g][0]]),#/len([g for g in gs if resDict[gs[g][1]][g][0]>0]),
         )       
#stdev of observed effect in combination treatment for upregulated vs downregulated?
#Proportion that change sign
#Mean reduction up/dn for each combo

In [ ]:
topG = (set([nmDict[t] for t in tpcm if t in nmDict and nmDict[t] not in classicTOP]),[g for g in classicTOP])

pp = PdfPages(outdir+"TOPcandidateRA.pdf")
gIdx = ['mouse_homologs','classic_TOP']
cIdx = ['blue','green']
plt.figure(figsize=(6,4))
#dat = [[resDict[c][g][0] for g in resDict[c] if g in resDict[c] and resDict[c][g][-1]<0.05] for c in ['PP242','AZD','BKM','MNK1','4EGI1']]#,'PP_6','PP_30']]#[[resDict[c][g][0] for c in ['PP242','AZD','BKM','MNK1','4EGI1'] if g in resDict[c] and resDict[c][g][-1]<0.05] for g in gset]
#sns.stripplot(data=dat,color='black',size=4)
rnks = ['PP242','AZD','BKM','MNK1','4EGI1']
for i,gset in enumerate(topG):
    print(len(gset))
    for c in rnks:#,'PP_6','PP_30']:
        print(c,len([g for g in gset if g in resDict[c] and resDict[c][g][-1]<0.05]))
    dat = [[resDict[c][g][0] for g in gset if g in resDict[c] and resDict[c][g][-1]<0.05] for c in rnks]#[[resDict[c][g][0] for c in ['PP242','AZD','BKM','MNK1','4EGI1'] if g in resDict[c] and resDict[c][g][-1]<0.05] for g in gset]
    sns.stripplot(data=dat,color=cIdx[i],alpha=0.5,size=6,edgecolors='none')
plt.xticks(range(len(rnks)),rnks)
handles = [mpatches.Patch(color=cIdx[i],label=gIdx[i]) for i in range(2)]
#plt.ylim(-2.7,0)
plt.legend(handles=handles,loc='lower left')
plt.ylabel("Log-Fold Change in RA, vs DMSO (p<0.05)")
plt.xlabel("Drug Tested")
plt.title("Changes in Ribosome Association across TOP Genes & Candidates by Drug")
pp.savefig()
plt.show()
pp.close()

In [ ]:
#ERCC depletion histograms for riboPLATEseq experiment
ddir = '/home/ubuntu/datadrive/riboPLATEseq/DESeq2/'
mod = 'tbasic'
norm = 'normB'
pfx = 'riboPLATE_ERCC'
fl = wDir+'{p}.final.{md}.{norm}.txt'.format(p=pfx,md=mod,norm=norm)
allcts = readMatrix(fl)
ctDict = {s:{g:float(v) for (g,v) in allcts[s].items()} for s in allcts}
riboD = {int(s.split('_')[1]):v for (s,v) in ctDict.items() if 'ribo' in s}                    
rnaD = {int(s.split('_')[1]):v for (s,v) in ctDict.items() if 'rna' in s}
sampsA = sorted(list(set([e for e in riboD.keys()])|set([e for e in rnaD.keys()])))
print(len(sampsA))
teD = {k:{v:((float(riboD[k][v])+1)/(float(rnaD[k][v])+1)) for v in riboD[k] if v in rnaD[k]} for k in riboD if k in rnaD}
sampsT = [k for k in teD.keys()]
ggenes = [g for g in teD[sampsT[0]].keys()]# if g in sigG[0]+sigG[1]]               
ercs = [g for g in ggenes if 'ERCC-' in g]
print(len(ercs))
ss = [i for i in range(96)[::2]]

pp = PdfPages(outdir+'ERCCdepletion_sum.pdf')
_,bns,_ = plt.hist([((sum([riboD[i][g] for g in ggenes if g not in ercs and rnaD[i][g]!=0]))/(sum([rnaD[i][g] for g in ggenes if g not in ercs and rnaD[i][g]!=0]))) for i in ss],bins=25,range=(0,2.0),alpha=0.5,color='blue',label='genome')
plt.hist([((sum([riboD[i][g] for g in ercs if g in riboD[i] and rnaD[i][g]!=0])/(sum([rnaD[i][g] for g in ercs if g in rnaD[i] and rnaD[i][g]!=0])))) for i in ss],bins=bns,alpha=0.5,color='red',label='spike-in')
plt.title("RA ratios in ERCC spike-in vs genes, summed")
plt.ylabel("Wells")
plt.xlabel("RA (sum(riboPLATE)/sum(RNA PLATE))")
plt.legend()
pp.savefig()
plt.show()
pp.close()

#nmG = [g for g in ggenes if g in nmDict.values()]
ercG = [g for g in ggenes if g in ercs]
nrcG = [g for g in ggenes if g not in ercs]
#Sum RA for non-ERCCs
print(np.mean([np.log2((sum([riboD[i][g] for g in ggenes if g not in ercs and rnaD[i][g]!=0]))/(sum([rnaD[i][g] for g in ggenes if g not in ercs and rnaD[i][g]!=0]))) for i in ss]))
#Sum RA for ERCCs
print(np.mean([np.log2((sum([riboD[i][g] for g in ggenes if g in ercs and rnaD[i][g]!=0]))/(sum([rnaD[i][g] for g in ggenes if g in ercs and rnaD[i][g]!=0]))) for i in ss]))
print(stats.mannwhitneyu([np.log2((sum([riboD[i][g] for g in ggenes if g not in ercs and rnaD[i][g]!=0]))/(sum([rnaD[i][g] for g in ggenes if g not in ercs and rnaD[i][g]!=0]))) for i in ss],
                         [np.log2((sum([riboD[i][g] for g in ggenes if g in ercs and rnaD[i][g]!=0]))/(sum([rnaD[i][g] for g in ggenes if g in ercs and rnaD[i][g]!=0]))) for i in ss],alternative='greater'))
print(stats.wilcoxon([np.log2((sum([riboD[i][g] for g in nrcG]))/(sum([rnaD[i][g] for g in nrcG]))) for i in ss],
                         [np.log2((sum([riboD[i][g] for g in ercG]))/(sum([rnaD[i][g] for g in ercG]))) for i in ss],correction=True,alternative='greater'))
print(np.mean([np.log2(np.mean([teD[i][g] for g in ggenes if g not in ercs and rnaD[i][g]!=0])) for i in ss]))
print(np.mean([np.log2(np.mean([teD[i][g] for g in ggenes if g in ercs and rnaD[i][g]!=0])) for i in ss]))
print(stats.mannwhitneyu([np.log2(np.mean([teD[i][g] for g in nrcG])) for i in ss],[np.log2(np.mean([teD[i][g] for g in ercG])) for i in ss],alternative='greater'))
print(stats.wilcoxon([np.log2(np.mean([teD[i][g] for g in nrcG])) for i in ss],[np.log2(np.mean([teD[i][g] for g in ercG])) for i in ss],correction=True,alternative='greater'))


##DIFFERENCE IN RA -- ERCC VS WHOLE GENOME--SUM (yields log2 RA depletion ratio)
pp = PdfPages(outdir+"ERCCdepletion_logsum.pdf")
_,bns,_ = plt.hist([np.log2((sum([riboD[i][g] for g in ercG if rnaD[i][g]!=0])/(sum([rnaD[i][g] for g in ercG if rnaD[i][g]!=0]))))-np.log2((sum([riboD[i][g] for g in nrcG if rnaD[i][g]!=0]))/(sum([rnaD[i][g] for g in nrcG if rnaD[i][g]!=0]))) for i in ss],bins=25)
plt.title("ERCC RA depletion ratios, summed")
plt.xlabel("log2(RA(ERCC/genome))")
plt.ylabel("Wells")
pp.savefig()
plt.show()
pp.close()
#Mean difference in all wells
print(np.mean([[np.log2((sum([riboD[i][g] for g in ercG])/(sum([rnaD[i][g] for g in ercG]))))-np.log2((sum([riboD[i][g] for g in nrcG]))/(sum([rnaD[i][g] for g in nrcG]))) for i in ss]]))# for i in ss]]))
#Mean difference excluding the two problematic wells
print(np.mean([[np.log2((sum([riboD[i][g] for g in ercG])/(sum([rnaD[i][g] for g in ercG]))))-np.log2((sum([riboD[i][g] for g in nrcG]))/(sum([rnaD[i][g] for g in nrcG]))) for i in ss if np.log2(sum([riboD[i][g] for g in ercG])/sum([rnaD[i][g] for g in ercG]))<0]]))# for i in ss]]))
print(stats.wilcoxon([np.log2((sum([riboD[i][g] for g in ercG])/(sum([rnaD[i][g] for g in ercG]))))-np.log2((sum([riboD[i][g] for g in nrcG]))/(sum([rnaD[i][g] for g in nrcG]))) for i in ss],correction=True,alternative='less'))
